# Applied Natural Language Processing 955G5
## Computer Based Examination, 2025

Remember, you can add cells and change their type (between code and text/markdown) as required to answer the questions.

In [20]:
# update your candidate number here
candidate_number = 11111111

# Question 2 (50 Marks)

This question is about semantics and distributional representations.

In [21]:
### do not change the code in this cell
# make sure you run this cell
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
from math import log, sqrt

stops = set(stopwords.words('english'))

sentences=[['A', 'semantic', 'network', ',', 'or', 'frame', 'network', 'is', 'a', 'knowledge', 'base', 'that', 'represents', 'semantic', 'connections', 'between', 'concepts', 'in', 'a', 'network', '.'],
           ['In', 'mathematics', 'and', 'computer', 'science', ',', 'graph', 'theory', 'is', 'the', 'study', 'of', 'graphs', ',', 'which', 'are', 'mathematical', 'structures', 'used', 'to', 'model', 'pairwise', 'relations', 'between', 'objects', '.'],
           ['A', 'neural', 'network', 'consists', 'of', 'connected', 'units', 'or', 'nodes', 'called', 'artificial', 'neurons', 'connected', 'by', 'edges', ',', 'which', 'model', 'the', 'synapses', 'in', 'the', 'brain', '.'],
           ['A', 'connectome', 'is', 'a', 'comprehensive', 'map', 'of', 'neural', 'connections', 'in', 'the', 'brain', ',', 'and', 'may', 'be', 'thought', 'of', 'as', 'its', "'wiring", 'diagram', "'", '.']]

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lukebirkett/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/lukebirkett/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lukebirkett/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


a) Build distributional representations of the words in `sentences` in the following steps.

i) Write a function that takes a list and an index as input and returns a list containing the two context tokens each side of that index, if they exist.

For example, given the list `["This", "is", "a", "sentence", "."]` and the index `1`, the function would return `["This", "a", "sentence"]`. But given the same list and the index `0` would return `["is", "a"]`.

(4 marks)

In [22]:
def window_slicer(lst, index):
    # 'window' is 2 as per the prompt "two tokens each side"
    left_context = lst[max(0, index - 2) : index]
    right_context = lst[index + 1 : index + 3]
    return left_context + right_context

example_sentence = ["This", "is", "a", "sentence", "."]
print(window_slicer(example_sentence, 0))
print(window_slicer(example_sentence, 1))
print(window_slicer(example_sentence, 2))
print(window_slicer(example_sentence, 3))
print(window_slicer(example_sentence, 4))
print(window_slicer(example_sentence, 5))

['is', 'a']
['This', 'a', 'sentence']
['This', 'is', 'sentence', '.']
['is', 'a', '.']
['a', 'sentence']
['sentence', '.']


ii) Using the function above, compute counts of token co-occurrences in the `sentences` list. The result should be a dictionary of dictionaries called `word_feature_counts`, with the keys of the outer dictionaries being the word tokens from `sentences` and the inner keys being the context token features returned by your function. For example, given the sentences `["This", "is", "an", "example"],["This", "is", "another"]]`, the tokens `This` and `is` co-occur twice. So, `word_feature_counts["is"]["This"]` would return the number `2`.

(8 marks)

In [41]:
def co_occurance(textlist):

    coocs = {}

    for sent in textlist:
        for index in range(len(sent)):

            token = sent[index]
            window_features = window_slicer(sent, index)

            current = coocs.get(token, {})

            for feature in window_features:
                current[feature] = current.get(feature, 0) + 1
            
            coocs[token] = current

    return coocs

word_feature_counts = co_occurance(sentences)
print(word_feature_counts["semantic"])


{'A': 1, 'network': 1, ',': 1, 'that': 1, 'represents': 1, 'connections': 1, 'between': 1}


b) So far, we have built a dictionary of word co-occurrences as an intial step towards constructing a distributional representation of semantics. Describe the alternative neural network approach as implemented in approaches such as CBOW.

(5 marks)

c)

i) Sum over the counts in `word_feature_counts` to produce a total count for all words and feature and a dictionary of total counts for each word, $w$, and a dictionary of total counts for each feature, $f$.

(5 marks)

* Word Count (dict)
* Feature Count (dict)
* Grand Total Count (value)

In [42]:
tokens = set([token for sentence in sentences for token in sentence])

words = {}
features = {}
total = 0

for word in tokens:
    current_features = word_feature_counts.get(word,{})
    words[word] = sum(current_features.values())

    for feature in current_features:
        features[feature] = features.get(feature,0)+current_features[feature]

total += sum(words.values())
total

356

ii) Turn these counts into Positive PMI values.

$$PPMI = \max \left( 0, \log \left( \frac{freq(w,f) \times freq_{tot}}{freq(w) \times freq(f)} \right) \right)$$

Here, $freq(w,f)$ represents the count in `word_feature_counts[w][f]` and $freq_{tot}$ is the sum of these counts for all words and features. $freq(w)$ is the total of the counts for word $w$ and $freq(f)$ is the total of the counts for feature $f$.

(5 marks)

In [46]:
import math
import copy

word_feature_ppmi = copy.deepcopy(word_feature_counts)
print(word_feature_ppmi)

for word in word_feature_counts.keys():
    nested_dict = word_feature_ppmi.get(word,{})
    for feature in nested_dict: 
        
        wf = nested_dict[feature]
        t = total

        w_freq = words[word]
        f_freq = features[feature]

        ppmi = max(0,math.log2((wf*t)/(w_freq*f_freq)))

        nested_dict[feature] = ppmi # direct reference update
    
print(word_feature_ppmi)

{'A': {'semantic': 1, 'network': 2, 'neural': 1, 'connectome': 1, 'is': 1}, 'semantic': {'A': 1, 'network': 1, ',': 1, 'that': 1, 'represents': 1, 'connections': 1, 'between': 1}, 'network': {'A': 2, 'semantic': 1, ',': 1, 'or': 2, 'frame': 1, 'is': 1, 'a': 2, 'in': 1, '.': 1, 'neural': 1, 'consists': 1, 'of': 1}, ',': {'semantic': 1, 'network': 1, 'or': 1, 'frame': 1, 'computer': 1, 'science': 1, 'graph': 1, 'theory': 1, 'of': 1, 'graphs': 1, 'which': 2, 'are': 1, 'by': 1, 'edges': 1, 'model': 1, 'the': 1, 'brain': 1, 'and': 1, 'may': 1}, 'or': {'network': 2, ',': 1, 'frame': 1, 'connected': 1, 'units': 1, 'nodes': 1, 'called': 1}, 'frame': {',': 1, 'or': 1, 'network': 1, 'is': 1}, 'is': {'frame': 1, 'network': 1, 'a': 2, 'knowledge': 1, 'graph': 1, 'theory': 1, 'the': 1, 'study': 1, 'A': 1, 'connectome': 1, 'comprehensive': 1}, 'a': {'network': 2, 'is': 2, 'knowledge': 1, 'base': 1, 'concepts': 1, 'in': 1, '.': 1, 'connectome': 1, 'comprehensive': 1, 'map': 1}, 'knowledge': {'is': 1,

iii) Define a function that will take two tokens and return their cosine similarity, based on the PPMI representation you have just constructed. Base your calculation on the expression below using dot products.

$$cos \left( w_1, w_2 \right) = \frac{w_1 \cdot w_2}{\sqrt{w_1 \cdot w_1 \times w_2 \cdot w_2}}$$

Apply this function to the tokens `"semantic"` and `"neural"`.

(7 marks)

In [49]:
def dot_product(v1, v2):
    dot = 0
    for feature, value in v1.items():
        dot += value * v2.get(feature, 0)
    return dot

def cosine_sim(v1, v2):
    numerator = dot_product(v1, v2)
    denominator = math.sqrt(dot_product(v1, v1) * dot_product(v2, v2))
    
    if denominator == 0:
        return 0
    return numerator / denominator

cosine_sim(
    word_feature_ppmi.get("semantic", {}),
    word_feature_ppmi.get("neural", {})
)

0.3609308711807981

d) Distributional representations are useful for identifying relations of semantic similarity between words. Name three other types of semantic relationship between words (such as might be found in WordNet) and give an example for each.

(6 marks)

Distribution represnetations are an unsupervised approach to semantic similarity between words. There are no input output labels, the information is contained and extracted from the input itself. On the other hands, we have Lexical Semantics which is a human-crafted, supervised approach, like that of WordNet which is a hierarchical dictionary. The structure of WordNet is built from 3, though not only 3, lexical semantics known as Synonyms, Hypernyms and Hyponyms. 

A Synonym is a word that means the exact same as another. A true Synonym can be used in place of another and not change the meaning of the sentence, though there may be contexual clues about meaning hidden in usage of a particular Synonym. 

Hypernyms represent parent or root words. It is the broader, general category of a collection of Synonyms (synset in WordNet). An example would be Feline is the Hypernym of Cat, Pather, Lion, etc. 

Hyponym is the inverse of this. It is the most specific, finer grain category derived from the Hypernym. Hypernym and Hyponym are not synonymous as Feline is place of Lion would loose too much information. 

e)

i) Apply stop word removal to the original list of sentences, recompute token co-occurrence counts, convert these counts to PPMI values and recompute the similarity of `"semantic"` and `"neural"`.

(5 Marks)

In [58]:
filtered_sentences = [[w for w in sent if w.lower() not in stops] for sent in sentences]
print(filtered_sentences)

# CO-OCCURANCE
word_feature_counts = co_occurance(filtered_sentences)

# FREQS
tokens = set([token for sentence in filtered_sentences for token in sentence])

words = {}
features = {}
total = 0

for word in tokens:
    current_features = word_feature_counts.get(word,{})
    words[word] = sum(current_features.values())

    for feature in current_features:
        features[feature] = features.get(feature,0)+current_features[feature]

total += sum(words.values())

#PPMI
word_feature_ppmi = copy.deepcopy(word_feature_counts)

for word in word_feature_counts.keys():
    nested_dict = word_feature_ppmi.get(word,{})
    for feature in nested_dict: 
        
        wf = nested_dict[feature]
        t = total

        w_freq = words[word]
        f_freq = features[feature]

        ppmi = max(0,math.log2((wf*t)/(w_freq*f_freq)))

        nested_dict[feature] = ppmi
    
cosine_sim(
    word_feature_ppmi.get("semantic", {}),
    word_feature_ppmi.get("neural", {})
)

[['semantic', 'network', ',', 'frame', 'network', 'knowledge', 'base', 'represents', 'semantic', 'connections', 'concepts', 'network', '.'], ['mathematics', 'computer', 'science', ',', 'graph', 'theory', 'study', 'graphs', ',', 'mathematical', 'structures', 'used', 'model', 'pairwise', 'relations', 'objects', '.'], ['neural', 'network', 'consists', 'connected', 'units', 'nodes', 'called', 'artificial', 'neurons', 'connected', 'edges', ',', 'model', 'synapses', 'brain', '.'], ['connectome', 'comprehensive', 'map', 'neural', 'connections', 'brain', ',', 'may', 'thought', "'wiring", 'diagram', "'", '.']]


0.16441506233767836

ii) Explain the difference between the two cosine similarity values, with and without stopword removal, and discuss which approach is preferable as a measure of semantic similarity.

(5 Marks)